# File reduction
Since the original file is over 5 GB, pandas struggles to open it all at once on a typical laptop with 8 GB RAM. 
As a solution, this program cuts it into chunks, groups events by the second (we don't care about millisecond precision of the event) and saves a new file.

If you set the chunk size to your liking, make sure that we get the same amount of rows. 

In [1]:
from scipy import stats
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def decoupling_time(r):
    '''
    This will output in this format: 1970-08-22 19:25:00 
    '''
    s = str(r)
    year, month, day, hour, minute, second = s[:4], s[4:6], s[6:8], s[8:10], s[10:12], s[12:14]
    
    return '-'.join([year, month, day]) + ' ' + ':'.join([hour, minute, second])
#decoupling_time(20201117235750093)


In [3]:
filename = "Raw Data/touchevent.csv"
final_data = pd.DataFrame()
#different chunk sizes, one_hundred is for tests. 
five_million = 5*10**6
three_million = 3*10**6
one_hundred = 10**2
c = 0
for chunk in tqdm(pd.read_csv(filename, chunksize=three_million)): 
    #deep copy 
    temp = chunk[['userid','day']]
    
    #turning now time from raw to machine readable
    shortened_times = pd.to_datetime(chunk['timestamp'].apply(
        decoupling_time), yearfirst=True).dt.floor('s')
    temp.insert(2, 'timestamp', shortened_times)
    
    #grouping 
    df_grouped = temp.groupby(['userid', 'day','timestamp' ]).size()                                         
    df_grouped = df_grouped.reset_index(name='touches')
    
    #making the day machine readable
    df_grouped.day = df_grouped.day.apply(lambda x: str(x)[:4]+'/'+str(x)[4:6]+'/'+str(x)[6:])
    df_grouped.day = pd.to_datetime(df_grouped.day, yearfirst=True)
    
    # ONLY NOVEMBER 
    '''
    df_grouped.day = df_grouped.day.apply(lambda x: True if str(x)[4:6] == "11" else False) #only november 
    df_grouped.rename(columns={"day": "first_two_weeks"}, inplace=True)
    df_grouped = df_grouped[df_grouped.first_two_weeks]   #this does the selection
    '''
    #df_grouped = df_grouped.drop(['first_two_weeks'], axis='columns')
    
    #appending to a global df
    final_data = pd.concat([final_data, df_grouped], ignore_index=True) 
    c += 1  #break during tests
    if c == 100:
        break
print(f"There were originally about {c*three_million:,} rows. Now there are exactly {final_data.shape[0]:,}")
print("File reduction is completed.")
final_data.to_csv('Processed Data/touch_sensor_unclean.csv')
print('The file was saved onto the current directory.')

20it [02:44,  8.03s/it]/tmp/ipykernel_37307/703943581.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  shortened_times = pd.to_datetime(chunk['timestamp'].apply(
44it [05:59,  8.17s/it]


There were originally about 132,000,000 rows. Now there are exactly 19,859,174
File reduction is completed.
The file was saved onto the current directory.


In [ ]:
# To run this, most of this cell has to be unblocked
# and needs to be put hashtags all over the "ONLY FIRST TWO WEEKS" block in the previous for loop.

days_in_total = final_data.day.value_counts().sort_index()
raw_data_days_frequency.to_csv('Processed Data/total_days_and_frequency.csv')
table=days_in_total
#table = pd.read_csv("Processed Data/total_days_and_frequency.csv").sort_index()
g = sns.barplot(data=table, x=table['day'], y=table['count'], color='#977dc7')
g.set_xticklabels(labels=table.day.apply(lambda x: str(x)[-2:] + '/' + str(x)[-5:-3]),rotation=90)
# It seems like from 12th of November 'till 11th of December there was most activity. 
plt.show()

# Make sure there are no mistakes.
ATTENTION: ONLY DO THIS IF YOU SET chunk size SMALLER THAN THREE MILLION ROWS

When grouping by second, because of the massive size of the original dataset, there can be mistakes when passing from one iteration to the other. For example, two observations with the same second but parsed in two different chunks for chance; this means that they don't get grouped in the same chunk and it will result in two different rows. 
The fastest solution is to check if the count of rows is right, then re-run the code on the parsed, processed new file as this time no _for_ loop is needed.

In [2]:
%time
filename = 'Processed Data/touch_sensor_unclean.csv'
by_second_unclean = pd.read_csv(filename)
by_second_unclean.drop('Unnamed: 0', axis='columns')

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.63 µs


In [8]:
def decoupling_time1(r):
    '''
    This will output in this format: 1970-08-22 19:25:00 
    '''
    s = str(r)
    year, month, day, hour, minute, second = s[:4], s[5:7], s[8:10], s[8:10], s[11:13], s[14:16]
    
    return '-'.join([year, month, day]) + ' ' + ':'.join([hour, minute, second])
decoupling_time1("1970-08-22 19:25:00")

'1970-08-22 22:19:25'

In [19]:
if by_second_unclean.shape[0] <= 19859174: #it should be this long, exactly every row; 
    #I have the solution in the case it's longer. 
    None #the count is right. 
elif by_second_unclean.shape[0] > 19859174:   
    temp = by_second_unclean[['userid','day']]
    shortened_times = pd.to_datetime(chunk['timestamp'].apply(decoupling_time1), 
                                     yearfirst=True).dt.floor('s')
    
    temp.insert(2, 'timestamp', shortened_times)
    
    df_grouped = temp.groupby(['userid', 'day', 'timestamp']).size()                                         
    df_grouped = df_grouped.reset_index(name='touches')
# I have no solution if it's shorter. Play with the chunk size until you get the correct result

In [24]:
print('outcome:')
try:
    print(f"{df_grouped.shape[0]:,}")
    df_grouped.to_csv(filename)
except NameError: 
    print(f"{by_second_unclean.shape[0]:,}")

outcome:
19,859,174
